In [50]:
import json
import time
import requests
import hashlib
import csv
import pandas as pd
from datetime import datetime


In [51]:
class DomClickApi:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({"X-Service": "true",
                                    "Connection": "Keep-Alive",
                                     "User-Agent": "Android; 12; Google; google_pixel_5; 8.72.0; 8720006; ; NONAUTH"
                                     })

        # init (get cookies)
        self.get("https://api.domclick.ru/core/no-auth-zone/api/v1/ensure_session")
        self.get("https://ipoteka.domclick.ru/mobile/v1/feature_toggles")

    def get(self, url, **kwargs):
        self.__update_headers(url, **kwargs)
        result = self.session.get(url, **kwargs)
        #print(self.session.cookies.get_dict())
        return result

    def __update_headers(self, url, **kwargs):
        url = self.__get_prepared_url(url, **kwargs)
        sault = "ad65f331b02b90d868cbdd660d82aba0"
        timestamp = str(int(datetime.now().timestamp()))
        encoded = (sault + url + timestamp).encode("UTF-8")
        h = hashlib.md5(encoded).hexdigest()
        self.session.headers.update({"Timestamp": timestamp,
                                     "Hash": "v1:" + h,
                                     })

    def __get_prepared_url(self, url, **kwargs):
        p = requests.models.PreparedRequest()
        p.prepare(method="GET", url=url, **kwargs)
        return p.url



In [52]:

def pprint_json(json_str):
    try:
        json_object = json.loads(json_str)
        json_formatted_str = json.dumps(json_object, indent=2, ensure_ascii=False).encode('utf8')
        print(json_formatted_str.decode())
    except:
        pass
        print(json_str)


In [53]:
def get_keys(d):
    if isinstance(d, dict):
        for k, v in d.items():
            yield k
            yield from list(get_keys(v))
    elif isinstance(d, list):
        for o in d:
            yield from list(get_keys(o))

In [54]:
def get_guid_of_regione(list_of_region) -> list:
        address = []
        for regione in list_of_region:
                offers_url = 'https://geo-service.domclick.ru/research/api/v1/autocomplete/regions'
                dca = DomClickApi()
                res = dca.get(offers_url, params={
                        "name": regione  #<<-- you regione str
                })
                count_obj = json.loads(res.text)
                address.append(count_obj['answer']['items'][0]['guid'])
        return address

   

In [66]:
from DB import tobd
def main_parser_fn(addresse):    
    offset = 0
    df_data = []
    remont = ['design', 'without_repair', 'standard', 'well_done'] #without_repair design standard well_done
    balcons = [1, 2]
    rooms = [1, 2, 3]
    type_dome = ['layout', 'flat'] #flat loyout
    perec = ['panel', 'brick', 'panel', 'monolith'] #monolith  brick panel block

    offers_url = 'https://offers-service.domclick.ru/research/v5/offers/'
    count_url = 'https://offers-service.domclick.ru/research/v5/offers/count/'

    dca = DomClickApi()
    for type in perec:
        for flat in rooms:
            for sd in type_dome:
                for rem in remont:
                    for balcon in balcons:
                        req = dca.get(count_url, params={
                            "address": addresse,
                            "deal_type": "sale",
                            "category": "living",
                            "offer_type": [sd],
                            "rooms": flat,
                            'renovation': rem,
                            "wall_type": type,
                            "balconies": balcon
                        })

                        count_obj = json.loads(req.text)
                        #print(count_obj)
                        total = count_obj["pagination"]["total"]
                        
                        for offset in range(0, total, 1):
                            try:
                                    
                                
                                res = dca.get(offers_url, params={
                                            "address": addresse,
                                            "deal_type": "sale",
                                            "category": "living",
                                            "offer_type": [sd],
                                            "offset": offset,
                                            "limit": 10,
                                            "rooms": flat,
                                            "wall_type": type,
                                            "renovation": rem,
                                            'balconies': balcon
                                })
                                print("RES:", res)
                                #pprint_json(res.text)
                                offers_obj = json.loads(res.text)
                                #print(list(get_keys(offers_obj)))
                                total = offers_obj["pagination"]["total"]
                                result_data = offers_obj['result']#['items']
                                items = result_data['items']
                                for item in items:
                                    address = item['address']
                                    price = item['price_info']
                                    house = item['house']
                                    object_info = item['object_info']
                                            #print(offers_obj)
                                            #renovation = item['renovation']
                                    row = (
                                                    #'address_id': address['id'],
                                        address['name'],
                                        price['price'],
                                        price['square_price'],
                                        object_info['floor'],
                                        #house['floors'],
                                        #'chat_avail': item['chat_available'],
                                        object_info['rooms'],
                                        object_info['area'],
                                        address['locality']['name'],
                                    #'type_house': type,
                                        sd,
                                        rem,
                                        balcon,
                                    #'year': result_data['items'][0]

                                    )
                                                # Находим seo_uri для данной станции метро
                                                #for seo_subway in seo_info['subways']:
                                                #    if seo_subway['guid'] == subway['guid']:
                                                #        row['seo_uri'] = seo_subway['seo_uri']
                                                #        break
                                            # Добавляем словарь в список данных
                                    tobd(row)
        #                           
                            except ConnectionError:
                                print("error")
    return 0                         
    #df = df.drop_duplicates(subset=['address_name']

In [67]:
addr = get_guid_of_regione(['Москва'])
for i in addr:
    main_parser_fn(addr)

RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Resp

KeyboardInterrupt: 

In [59]:
m = pd.read_csv('12.csv')
m

,Unnamed: 0,address_name,price,square_price,floors,total_floors,rooms,area,local,type_home,remont,balcon
0,0,"Ленинградское шоссе, 228к6",8950000,236147,10,12,1,37.9,Москва,layout,without_repair,1
1,1,"Ленинградское шоссе, 228к4",10500000,250000,4,14,1,42.0,Москва,layout,without_repair,1
2,2,"Ленинградское шоссе, 228к7",9900000,396000,4,15,1,25.0,Москва,layout,without_repair,1
3,3,"жилой комплекс Новое Очаково, к1.4",12500000,345303,22,33,1,36.2,Москва,layout,without_repair,1
4,4,"Ленинградское шоссе, 228к6",8150000,237609,5,15,1,34.3,Москва,layout,standard,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11266,11266,"Новоясеневский проспект, 19к1",16000000,274914,3,9,3,58.2,Москва,flat,well_done,2
11267,11267,"улица Коминтерна, 12",25500000,279299,17,17,3,91.3,Москва,flat,well_done,2
11268,11268,"Зеленоград, к829",21000000,267175,5,19,3,78.6,Зеленоград,flat,well_done,2
11269,11269,"шоссе Энтузиастов, 11Ак4",35500000,473333,3,17,3,75.0,Москва,flat,well_done,2


In [ ]:
df = pd.read_csv('data/1_data.csv')
df['d'] = df['type_home'].replace(['loyout', 'flat'], [0,1])
df

In [ ]:
cor = df['price'].corr(df['d'])
cor

In [5]:
main = pd.read_csv('data/1_data.csv')
#main = pd.concat([df, main], ignore_index=True)
#main.to_csv('1_data.csv')

In [737]:
#main = main.drop_duplicates(subset=['address_name'])

In [14]:
main = main.drop(['Unnamed: 0.58'], axis = 1, )

In [16]:

main.to_csv('1_data.csv')


In [17]:
main

,address_name,price,square_price,floors,rooms,area,local,type_house,type_home,remont,total_floors,balcon
0,"улица Грекова, 3к2",22500000,350467,16,3,64.20,Москва,panel,loyout,design,16,2
1,"Береговой проезд, 9к5",17600000,451282,12,1,39.00,Москва,panel,loyout,design,12,2
2,"Сивашская улица, 7",36500000,325892,17,4,112.00,Москва,panel,loyout,design,17,2
3,"Окская улица, 5к1",44999999,409090,23,4,110.00,Москва,panel,loyout,design,23,2
4,"Дубнинская улица, 2к1",16499000,260812,12,3,63.26,Москва,panel,loyout,design,12,2
...,...,...,...,...,...,...,...,...,...,...,...,...
35470,"Детский проезд, 1к1",6400000,118518,19,2,54.00,Казань,monolith,flat,standard,19,1
35471,"Нижняя улица, 8",7800000,95823,17,3,81.40,Казань,monolith,flat,standard,17,1
35472,"улица Рауиса Гареева, 109",5650000,200996,23,0,28.11,Казань,monolith,flat,standard,23,1
35473,"улица Нурихана Фаттаха, 17/4",6000000,92307,18,2,65.00,Казань,monolith,flat,standard,18,1
